# Imports and Const

In [1]:
import os
import pandas as pd

where is the data located

In [2]:
based_data_path = '/home/eyal/Desktop/University/Research/DeepWithdrawn/split/'

# Prep Data

This step converts our data to the format t5chem expects to receive

In [3]:
def load_data(scheme, modal):
    train_data_path = os.path.join(based_data_path, scheme, modal, 'train2.csv')
    val_data_path = os.path.join(based_data_path, scheme, modal, 'val.csv')
    test_data_path = os.path.join(based_data_path, scheme, modal, 'test.csv')
    
    train_df = pd.read_csv(train_data_path, usecols=['smiles', 'withdrawn_class'])
    train_df['type'] = 'train'

    val_df = pd.read_csv(val_data_path, usecols=['smiles', 'withdrawn_class'])
    val_df['type'] = 'val'

    test_df = pd.read_csv(test_data_path, usecols=['smiles', 'withdrawn_class'])
    test_df['type'] = 'test'

    raw_df = pd.concat([train_df, val_df, test_df])
    
    return raw_df
    

In [4]:
def prepare_input(row, source_file, target_file):
    print(row['smiles'], file=source_file)
    print(row['withdrawn_class'], file=target_file)

In [5]:
for folder in ['db_agree_dups', 'db_agree_no_dups', 'db_no_agree_dups', 'db_no_agree_no_dups']:
    print(folder)
    for modal in ['DrugBank', 'ChEMBL', 'NCATS']:
        raw_df = load_data(folder, modal)
        input_source_dict = {}
        input_target_dict = {}
        input_folder_path = os.path.join("./data/drug_withdrawn", folder, modal)
        os.makedirs(input_folder_path)
        for data_type in ['train', 'val', 'test']:
            input_source_dict[data_type] = open(os.path.join(input_folder_path, data_type + '.source'), "w")
            input_target_dict[data_type] = open(os.path.join(input_folder_path, data_type + '.target'), "w")
        for i,item in raw_df.iterrows():
            data_type = item['type']
            prepare_input(item, input_source_dict[data_type], input_target_dict[data_type])

        for key, val in input_source_dict.items():
            val.close()
        for key,val in input_target_dict.items():
            val.close()

db_agree_dups
db_agree_no_dups
db_no_agree_dups
db_no_agree_no_dups


# Train

here we point to the path of the pre-trained model
and then train t5chem for each evaluation and each test dataset

In [7]:
pre_train_model = './models/pretrain/simple/'

In [ ]:
for folder in ['db_agree_dups', 'db_agree_no_dups', 'db_no_agree_dups', 'db_no_agree_no_dups']:
    for modal in ['DrugBank', 'ChEMBL', 'NCATS']:
        input_folder_path = os.path.join("./data/drug_withdrawn", folder, modal)
        output_folder_path = os.path.join("./model", folder, modal)
        train = "t5chem train --data_dir %s --output_dir %s --num_classes %s --pretrain %s --task_type classification --num_epoch %s" % (input_folder_path, output_folder_path, 2, pre_train_model, 100) 

        os.system(train)



Some weights of T5ForProperty were not initialized from the model checkpoint at ./models/pretrain/simple/ and are newly initialized: ['lm_head.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/eyal/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4341
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 13600
  Number of trainable parameters = 14840308
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged 

Namespace(data_dir='./data/drug_withdrawn/db_agree_dups/DrugBank', output_dir='./model/db_agree_dups/DrugBank', task_type='classification', pretrain='./models/pretrain/simple/', vocab='', tokenizer='', random_seed=8570, num_epoch=100, log_step=5000, batch_size=32, init_lr=0.0005, num_classes=2, command=<function train at 0x7f2406e2d510>)


  0%|          | 42/13600 [00:10<58:53,  3.84it/s]  
wandb: Waiting for W&B process to finish... (success).
wandb: Synced ./model/db_agree_dups/DrugBank: https://wandb.ai/eyalmazuz/huggingface/runs/2bz3owic
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20221204_171307-2bz3owic/logs
Some weights of T5ForProperty were not initialized from the model checkpoint at ./models/pretrain/simple/ and are newly initialized: ['lm_head.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/eyal/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4169
  Num Epochs 

Namespace(data_dir='./data/drug_withdrawn/db_agree_dups/ChEMBL', output_dir='./model/db_agree_dups/ChEMBL', task_type='classification', pretrain='./models/pretrain/simple/', vocab='', tokenizer='', random_seed=8570, num_epoch=100, log_step=5000, batch_size=32, init_lr=0.0005, num_classes=2, command=<function train at 0x7f6751321510>)


  0%|          | 22/13100 [00:06<1:01:44,  3.53it/s]
wandb: Waiting for W&B process to finish... (success).
wandb: Synced ./model/db_agree_dups/ChEMBL: https://wandb.ai/eyalmazuz/huggingface/runs/gwq8x215
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20221204_171335-gwq8x215/logs


# Predict

here we predict and evaluate all the metrics for our t5chem model for all split and test datasets

In [3]:
pre_train_model = './models/pretrain/simple/'

In [4]:
for folder in ['db_agree_dups', 'db_agree_no_dups', 'db_no_agree_dups', 'db_no_agree_no_dups']:
    for modal in ['DrugBank', 'ChEMBL', 'NCATS']:
        input_folder_path = os.path.join("./data/drug_withdrawn", folder, modal)
        model_folder_path = os.path.join("./model", folder, modal)
        pred_output_path = os.path.join("./model", folder, modal, 'raw_prob.csv')
        predict = 't5chem predict --data_dir %s --model_dir %s --prediction %s' % (input_folder_path, model_folder_path, pred_output_path)
        
        os.system(predict)
        

sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found
sh: line 1: t5chem: command not found


In [13]:
def load_test(model_path):
    df = pd.read_csv(os.path.join(model_path, 'preds.csv'), index_col=0)
    return df

In [18]:
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, auc, average_precision_score
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, roc_auc_score, average_precision_score, accuracy_score

def get_scores(df):
    y = np.array(df['target'].tolist())
    pred = np.array(df['prob'].tolist())
    fpr, tpr, thresholds = roc_curve(y, pred)
    precision, recall, thresholds = precision_recall_curve(y, pred)
    mcc = matthews_corrcoef(y_true=y, y_pred=(pred > 0.5))
    tn, fp, fn, tp = confusion_matrix(y_true=y, y_pred=(pred > 0.5)).ravel()
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp+fn)
    roc = roc_auc_score(y, pred)
    aupr = average_precision_score(y, pred)
    acc = accuracy_score(y_true=y, y_pred=(pred > 0.5))
    return {'auroc': auc(fpr, tpr),'auprc': aupr,'roc': roc,'acc': acc,'mcc': mcc,'sp': specificity,'sn': sensitivity}


In [21]:
final_scores_df = pd.DataFrame(columns=['auprc', 'auroc', 'roc', 'acc', 'mcc', 'sp', 'sn', 'folder', 'modal'])
for folder in ['db_agree_no_dups', 'db_no_agree_dups']:
    for modal in ['ChEMBL', 'DrugBank','NCATS']:
        input_folder_path = os.path.join("./model", folder, modal)
        df = load_test(input_folder_path)
        
        dic = get_scores(df)
        dic.update({'folder': folder, 'modal': modal})
        final_scores_df.loc[len(final_scores_df)] = pd.Series(dic)

In [22]:
final_scores_df

,auprc,auroc,roc,acc,mcc,sp,sn,folder,modal
0,0.600002,0.778345,0.778345,0.871948,0.479812,0.961219,0.431694,db_agree_no_dups,ChEMBL
1,0.389629,0.804716,0.804716,0.336024,0.128098,0.282489,0.921687,db_agree_no_dups,DrugBank
2,0.42543,0.688401,0.688401,0.677312,0.235812,0.714749,0.551932,db_agree_no_dups,NCATS
3,0.869888,0.923874,0.923874,0.906823,0.734971,0.962032,0.732143,db_no_agree_dups,ChEMBL
4,0.768516,0.910294,0.910294,0.95146,0.719035,0.967902,0.785388,db_no_agree_dups,DrugBank
5,0.54772,0.772386,0.772386,0.705706,0.360215,0.715952,0.678506,db_no_agree_dups,NCATS


In [38]:
{'hello': 1}.update({'world':2})

In [43]:
from transformers import T5ForConditionalGeneration
from t5chem import T5ForProperty, SimpleTokenizer
import os
import torch
from tqdm.auto import tqdm
sm = torch.nn.Softmax(dim=1)

final_scores_df = pd.DataFrame(columns=['auprc', 'auroc', 'roc', 'acc', 'mcc', 'sp', 'sn', 'folder', 'modal'])
for folder in ['db_agree_no_dups', 'db_agree_dups']:
    for modal in ['ChEMBL', 'DrugBank',' NCATS']:
        input_folder_path = os.path.join("./data/drug_withdrawn", folder, modal)
        model_folder_path = os.path.join("./model", folder, modal)
        tokenizer = SimpleTokenizer(vocab_file=os.path.join(model_folder_path, 'vocab.pt'))
        model = T5ForProperty.from_pretrained(model_folder_path)  # for non-seq2seq task
        df = load_test(input_folder_path, model_folder_path)
        probs = []
        for i, row in tqdm(df.iterrows()):
            inputs = tokenizer.encode("Classification:" + row['smiles'], return_tensors='pt')
            outputs = model(inputs)
            probs.append(sm(outputs.logits)[0][-1].detach().numpy().item())
        df['prob'] = probs
        df.to_csv(os.path.join(model_folder_path, 'preds.csv'))
        dic = get_scores(df)
        dic.update({'folder': folder, 'modal': modal})
        final_scores_df.loc[len(final_scores_df)] = pd.Series(dic)
#         final_scores_df = pd.concat([final_scores_df, pd.DataFrame([pd.Series(dic)])], ignore_index=True)
#         final_scores_df = final_scores_df.append(pd.Series(dic), ignore_index=True)

In [44]:
final_scores_df

,auprc,auroc,folder,modal
0,0.868661,0.938558,db_agree_dups,DrugBank
1,0.876535,0.933469,db_agree_dups,ChEMBL
2,0.530063,0.783790,db_agree_dups,NCATS
3,0.389629,0.804716,db_agree_no_dups,DrugBank
4,0.600002,0.778345,db_agree_no_dups,ChEMBL
5,0.425430,0.688401,db_agree_no_dups,NCATS
6,0.768516,0.910294,db_no_agree_dups,DrugBank
7,0.869888,0.923874,db_no_agree_dups,ChEMBL
8,0.547720,0.772386,db_no_agree_dups,NCATS
9,0.364341,0.771137,db_no_agree_no_dups,DrugBank


In [45]:
final_scores_df.to_csv('./T5chem_final_results.csv', index=False)